In [1]:
!pip -q install huggingface transformers boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 4.2 MB/s eta 0:00:00


In [2]:
import os
import sys
import shutil
import tarfile
import argparse
import boto3
import torch
from transformers import AutoTokenizer, GPTJForCausalLM

In [3]:
def compress(tar_dir=None, output_file="model.tar.gz"):
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(tar_dir, arcname=os.path.sep)

In [12]:
def upload_file_to_s3(bucket_name=None, file_name="model.tar.gz", key_prefix=""):
    s3 = boto3.resource("s3",aws_access_key_id='',
    aws_secret_access_key='')
    key_prefix_with_file_name = os.path.join(key_prefix, file_name)
    s3.Bucket(bucket_name).upload_file(file_name, key_prefix_with_file_name)
    return f"s3://{bucket_name}/{key_prefix_with_file_name}"

In [5]:
model_save_dir = "./tmp"
key_prefix = "gpt-j"
src_inference_script = os.path.join("code", "inference.py")
dst_inference_script = os.path.join(model_save_dir, "code")


In [6]:
os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(dst_inference_script, exist_ok=True)

In [ ]:
# load fp 16 model
print("Loading model from `EleutherAI/gpt-j-6B`")
model = GPTJForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16
)


In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [8]:
print("saving model with `torch.save`")
torch.save(model, os.path.join(model_save_dir, "benderbot.pt"))


saving model with `torch.save`


In [9]:
print("saving tokenizer")
tokenizer.save_pretrained(model_save_dir)


saving tokenizer


('./tmp/tokenizer_config.json',
 './tmp/special_tokens_map.json',
 './tmp/vocab.txt',
 './tmp/added_tokens.json',
 './tmp/tokenizer.json')

In [ ]:
# copy inference script
print("copying inference.py script")
shutil.copy(src_inference_script, dst_inference_script)

In [10]:
# create archive
print("creating `model.tar.gz` archive")
compress(model_save_dir)


creating `model.tar.gz` archive


In [13]:
# upload to s3
print(
    f"uploading `model.tar.gz` archive to s3"
)

model_uri = upload_file_to_s3(bucket_name='boto-bucket-16', 
                              key_prefix=key_prefix,
                              )

print(f"Successfully uploaded to {model_uri}")

uploading `model.tar.gz` archive to s3
Successfully uploaded to s3://boto-bucket-16/gpt-j/model.tar.gz
